In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from parsel import Selector 
import requests
import lxml.html as lh
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options  

In [30]:
latitude = -24.3743
longitude = 133.4159
import time
from datetime import date 

def analyse(gfs = True):
    today = date.today()

    current_month = today.month
    current_day = today.day

    altitude_params = ["", "100m", "950h", "925h", "900h", "850h", "800h", "700h",
                      "600h", "500h", "400h", "300h", "250h", "200h", "150h"]

    altitudes = [100, 300, 600, 750, 900, 1500, 2000, 3000, 4200, 5500, 7000,
                 9000, 10000, 11700, 13500]

    driver = webdriver.Chrome('/usr/local/bin/chromedriver')

    storage = dict()

    import time

    start_time = time.time()

    from_date = 31
    to_date = 32

    # Check next 7 days
    for i in range(from_date - 1, to_date -1):
        day = i
        if (day <= 30):
            date_key = f"{day+1} May"

        else: 
            date_key = f"{day-31+1} June"

        print(date_key)
        storage[date_key] = {}
        #Check from 5 am to 10am 
        for j in range(6, 8): #(5,11) 
            time_str = 15 + j

        # Reset the lsit     
            data = []
        # Altitudes
            k = 0
            while k < len(altitude_params): 
                print(f"k: {k}")
    #             for mode in modes: 
                if gfs:
                    if k == 0:
                        gfs_term = "gfs"
                    
                    else:
                        gfs_term = "gfs,"
                        
                    
                    url = f"https://www.windy.com/?{gfs_term}{altitude_params[k]},2019-05-{day}-{time_str},{latitude},{longitude},9,m:cZOajSs"
                else:
                    url = f"https://www.windy.com/?{altitude_params[k]},2019-05-{day}-{time_str},{latitude},{longitude},9,m:cZOajSs"
                print(url)
                driver.get(url)
                time.sleep(2.5)
                marker = driver.find_elements_by_tag_name('big')

                processed_marker = [e.text for e in marker]

                data_point = processed_marker[0]
                print(data_point)

                # Remove the rougue inverted commas
                data_point = data_point[1:]

                data_point_processed = data_point.split(' ')
                try: 
                    data.append((altitudes[k], data_point_processed[0], int(data_point_processed[1][:-2])))
                    k += 1    
                except: 
                    print("repeating")


            storage[date_key][f"{j} am"] = data



    print(f"time taken = {time.time() - start_time}")

    processed_storage = dict()

    for day, obj in storage.items(): 
        processed_storage[day] = dict()
        processed_storage[day]["altitude"] = set() 
        for time, arr in obj.items():

            processed_storage[day][time + " direction"] = list()
            processed_storage[day][time + " wind speed (Knots)"] = list()
            processed_storage[day]["altitude"] = [tup[0] for tup in arr]

            for tup in arr: 
                processed_storage[day][time + " direction"].append(tup[1])
                processed_storage[day][time + " wind speed (Knots)"].append(tup[2])

        # convert set to list
        processed_storage[day]["altitude"] = list(processed_storage[day]["altitude"])

    today_str = str(date.today())

    if gfs:
        path = f"./{today_str}_gfs_data.xlsx"
    else: 
        path = f"./{today_str}_data.xlsx"
        
    writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

    for day, day_data in processed_storage.items():
        print(day)
        df_temp = pd.DataFrame(day_data).set_index('altitude')
        if gfs:
            df_temp.to_excel(writer, sheet_name = day + "_gfs")

        else: 
            df_temp.to_excel(writer, sheet_name = day)

    writer.save()
    writer.close()


In [31]:
analyse(True)
analyse(False)

31 May
k: 0
https://www.windy.com/?gfs,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"SE 16kt
k: 1
https://www.windy.com/?gfs,100m,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 23kt
k: 2
https://www.windy.com/?gfs,950h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 21kt
k: 3
https://www.windy.com/?gfs,925h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 28kt
k: 4
https://www.windy.com/?gfs,900h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 32kt
k: 5
https://www.windy.com/?gfs,850h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"SE 27kt
k: 6
https://www.windy.com/?gfs,800h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 25kt
k: 7
https://www.windy.com/?gfs,700h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 19kt
k: 8
https://www.windy.com/?gfs,600h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 15kt
k: 9
https://www.windy.com/?gfs,500h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"E 11kt
k: 10
https://www.windy.com/?gfs,400h,2019-05-30-21,-24.3743,133.4159,9,m:cZOajSs
"S 5kt
k: 11
https://www

In [ ]:
processed_storage = dict()

for day, obj in storage.items(): 
    processed_storage[day] = dict()
    processed_storage[day]["altitude"] = set() 
    for time, arr in obj.items():
       
        processed_storage[day][time + " direction"] = list()
        processed_storage[day][time + " wind speed (Knots)"] = list()
        processed_storage[day]["altitude"] = [tup[0] for tup in arr]
        
        for tup in arr: 
            processed_storage[day][time + " direction"].append(tup[1])
            processed_storage[day][time + " wind speed (Knots)"].append(tup[2])
    
    # convert set to list
    processed_storage[day]["altitude"] = list(processed_storage[day]["altitude"])

print(processed_storage)

In [7]:
from datetime import date 
import pandas as pd
today_str = str(date.today())

path = f"./{today_str}_data.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

for day, day_data in processed_storage.items():
    print(day)
    df_temp = pd.DataFrame(day_data).set_index('altitude')
    df_temp.to_excel(writer, sheet_name = day + "(+1)")
    
writer.save()
writer.close()

-2 June
-1 June
0 June


In [8]:
from datetime import date 
str(date.today())

'2019-05-22'

In [9]:
print(storage)

{'-2 June': {'6 am': [(100, 'SE', 10), (300, 'SE', 12), (600, 'SE', 10), (750, 'SE', 17), (900, 'SE', 24), (1500, 'SE', 25), (2000, 'SE', 21), (3000, 'SW', 10), (4200, 'SW', 19), (5500, 'W', 35), (7000, 'W', 47), (9000, 'W', 69), (10000, 'W', 73), (11700, 'W', 94), (13500, 'W', 83)], '7 am': [(100, 'SE', 10), (300, 'SE', 12), (600, 'SE', 10), (750, 'SE', 17), (900, 'SE', 24), (1500, 'SE', 25), (2000, 'SE', 21), (3000, 'SW', 10), (4200, 'SW', 19), (5500, 'W', 35), (7000, 'W', 47), (9000, 'W', 69), (10000, 'W', 73), (11700, 'W', 94), (13500, 'W', 83)]}, '-1 June': {'6 am': [(100, 'SE', 18), (300, 'SE', 24), (600, 'SE', 19), (750, 'SE', 26), (900, 'SE', 30), (1500, 'SE', 30), (2000, 'SE', 28), (3000, 'E', 16), (4200, 'S', 5), (5500, 'SW', 6), (7000, 'W', 22), (9000, 'W', 44), (10000, 'W', 46), (11700, 'W', 49), (13500, 'W', 47)], '7 am': [(100, 'SE', 18), (300, 'SE', 24), (600, 'SE', 19), (750, 'SE', 26), (900, 'SE', 30), (1500, 'SE', 30), (2000, 'SE', 28), (3000, 'E', 16), (4200, 'S', 5)